In [1]:
import pandas as pd
import numpy as np
import json
import requests

In [2]:
raw = pd.read_csv("PropensityBlockModel.csv")
raw.head()

,GEOID,Max,Sum,house_yr_pct_earlier_1939,house_pct_occupied,house_pct_family_married,race_pct_black,worked_past_12_mo,heat_pct_fueloil_kerosene,educ_bachelors,house_pct_live_alone,educ_some_col_no_grad,house_pct_ownd_occupied,house_w_home_equity_loan,house_val_175K_200K,house_val_200K_250K,True_Labels,Predicted_Labels,Predicted_Probs
0,#_270170706003,1.508296,3.016591,0.259459,0.694595,0.513619,0.015060,0.619217,0.058366,0.071283,0.268482,0.197556,0.898833,0.038961,0.025974,0.086580,0,0,0.305971
1,#_261158315002,0.000000,0.000000,0.000000,0.878641,0.483425,0.004278,0.435175,0.000000,0.057007,0.334254,0.061758,0.762431,0.032609,0.047101,0.083333,0,0,0.128296
2,#_261390209002,0.000000,0.000000,0.804455,0.727723,0.489796,0.000000,0.647368,0.000000,0.493827,0.370748,0.080247,0.782313,0.108696,0.030435,0.130435,0,0,0.384859
3,#_420110016001,1.486989,1.486989,0.420147,0.916462,0.402145,0.023792,0.743702,0.335121,0.060820,0.193029,0.073550,0.512064,0.036649,0.000000,0.000000,0,0,0.350192
4,#_200050818003,2.210759,3.684598,0.735808,0.930131,0.570423,0.023581,0.646445,0.000000,0.256098,0.260563,0.131707,0.734742,0.019169,0.000000,0.000000,0,0,0.340038


In [3]:
raw['copy_index'] = raw.index
raw

,GEOID,Max,Sum,house_yr_pct_earlier_1939,house_pct_occupied,house_pct_family_married,race_pct_black,worked_past_12_mo,heat_pct_fueloil_kerosene,educ_bachelors,house_pct_live_alone,educ_some_col_no_grad,house_pct_ownd_occupied,house_w_home_equity_loan,house_val_175K_200K,house_val_200K_250K,True_Labels,Predicted_Labels,Predicted_Probs,copy_index
0,#_270170706003,1.508296,3.016591,0.259459,0.694595,0.513619,0.015060,0.619217,0.058366,0.071283,0.268482,0.197556,0.898833,0.038961,0.025974,0.086580,0,0,0.305971,0
1,#_261158315002,0.000000,0.000000,0.000000,0.878641,0.483425,0.004278,0.435175,0.000000,0.057007,0.334254,0.061758,0.762431,0.032609,0.047101,0.083333,0,0,0.128296,1
2,#_261390209002,0.000000,0.000000,0.804455,0.727723,0.489796,0.000000,0.647368,0.000000,0.493827,0.370748,0.080247,0.782313,0.108696,0.030435,0.130435,0,0,0.384859,2
3,#_420110016001,1.486989,1.486989,0.420147,0.916462,0.402145,0.023792,0.743702,0.335121,0.060820,0.193029,0.073550,0.512064,0.036649,0.000000,0.000000,0,0,0.350192,3
4,#_200050818003,2.210759,3.684598,0.735808,0.930131,0.570423,0.023581,0.646445,0.000000,0.256098,0.260563,0.131707,0.734742,0.019169,0.000000,0.000000,0,0,0.340038,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134503,#_110010050011,0.000000,0.000000,0.593137,0.926471,0.351852,0.070358,0.961735,0.000000,0.382843,0.365079,0.025608,0.497354,0.015957,0.000000,0.000000,0,0,0.000000,134503
134504,#_110010068041,0.000000,0.000000,0.851852,1.000000,0.296296,0.865349,0.487495,0.000000,0.037674,0.148148,0.116279,1.000000,0.148148,0.000000,0.000000,0,0,0.000000,134504
134505,#_110010070001,0.000000,0.000000,0.551389,0.948611,0.370425,0.078049,0.899194,0.000000,0.386930,0.387994,0.016337,0.522694,0.274510,0.000000,0.000000,0,0,0.000000,134505
134506,#_110010081003,0.000000,0.000000,0.895787,0.793792,0.455307,0.067775,0.751037,0.019553,0.431988,0.284916,0.017937,0.463687,0.156627,0.000000,0.000000,0,0,0.000000,134506


In [4]:


tracts = pd.read_csv(r'API/2019_Gaz_tracts_national.txt',sep = '\t',header = 0,names=['USPS','GEOID','ALAND','AWATER', 
                                                                               'ALAND_SQMI','AWATER_SQMI','INTPTLAT', 'INTPTLONG'])

tracts.to_csv(r'API/2019_Gaz_tracts_national.csv', index=None)



In [5]:
tracts

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338
1,AL,1001020200,3325680,5669,1.284,0.002,32.475758,-86.472468
2,AL,1001020300,5349273,9054,2.065,0.003,32.474024,-86.459703
3,AL,1001020400,6384276,8408,2.465,0.003,32.471030,-86.444835
4,AL,1001020500,11408866,43534,4.405,0.017,32.458922,-86.421826
...,...,...,...,...,...,...,...,...
73996,PR,72153750501,1820185,0,0.703,0.000,18.031211,-66.867347
73997,PR,72153750502,689930,0,0.266,0.000,18.024746,-66.860442
73998,PR,72153750503,3298433,1952,1.274,0.001,18.023148,-66.876603
73999,PR,72153750601,10987037,4527,4.242,0.002,18.017809,-66.839070


In [6]:
def tracts_(geoid):
    geoid = geoid.replace('#_', '')
    geoid = geoid.replace(' ', '')
    geoid = geoid[:-1]
    if geoid.startswith('0'):
        geoid= geoid.replace('0','', 1)
    else:
        pass
    return geoid

In [7]:
tracts_list = []

for index, i in enumerate(zip(raw['copy_index'], raw['GEOID'])):
    copy_index= i[0]
    GEOID = tracts_(i[1])
    
    tracts_list.append({'copy_index' : copy_index,
                       'GEOID': GEOID})

In [8]:
tracts_list

[{'copy_index': 0, 'GEOID': '27017070600'},
 {'copy_index': 1, 'GEOID': '26115831500'},
 {'copy_index': 2, 'GEOID': '26139020900'},
 {'copy_index': 3, 'GEOID': '42011001600'},
 {'copy_index': 4, 'GEOID': '20005081800'},
 {'copy_index': 5, 'GEOID': '12099001913'},
 {'copy_index': 6, 'GEOID': '4013116602'},
 {'copy_index': 7, 'GEOID': '51730811300'},
 {'copy_index': 8, 'GEOID': '36103158010'},
 {'copy_index': 9, 'GEOID': '34025808500'},
 {'copy_index': 10, 'GEOID': '42001030102'},
 {'copy_index': 11, 'GEOID': '17057953500'},
 {'copy_index': 12, 'GEOID': '6075022901'},
 {'copy_index': 13, 'GEOID': '22107000200'},
 {'copy_index': 14, 'GEOID': '6075026002'},
 {'copy_index': 15, 'GEOID': '17077010300'},
 {'copy_index': 16, 'GEOID': '18035002301'},
 {'copy_index': 17, 'GEOID': '40095094698'},
 {'copy_index': 18, 'GEOID': '17031804714'},
 {'copy_index': 19, 'GEOID': '9003416500'},
 {'copy_index': 20, 'GEOID': '6111001002'},
 {'copy_index': 21, 'GEOID': '37183053522'},
 {'copy_index': 22, 'GEOI

In [9]:
tracts_list = pd.DataFrame(tracts_list, columns = ['copy_index', 'GEOID'])
tracts_list

,copy_index,GEOID
0,0,27017070600
1,1,26115831500
2,2,26139020900
3,3,42011001600
4,4,20005081800
...,...,...
134503,134503,11001005001
134504,134504,11001006804
134505,134505,11001007000
134506,134506,11001008100


In [10]:
tracts['GEOID'] = tracts['GEOID'].astype(str)
tracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   USPS         74001 non-null  object 
 1   GEOID        74001 non-null  object 
 2   ALAND        74001 non-null  int64  
 3   AWATER       74001 non-null  int64  
 4   ALAND_SQMI   74001 non-null  float64
 5   AWATER_SQMI  74001 non-null  float64
 6   INTPTLAT     74001 non-null  float64
 7   INTPTLONG    74001 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 4.5+ MB


In [11]:
tracts_list = pd.merge(tracts_list, tracts, left_on='GEOID', right_on= 'GEOID', how = 'left')
tracts_list

,copy_index,GEOID,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,0,27017070600,MN,977465236,10306005,377.401,3.979,46.600099,-92.882993
1,1,26115831500,MI,4202063,64039,1.622,0.025,41.931336,-83.398945
2,2,26139020900,MI,4699929,1325661,1.815,0.512,43.064721,-86.213325
3,3,42011001600,PA,250449,0,0.097,0.000,40.340507,-75.913125
4,4,20005081800,KS,35296180,780343,13.628,0.301,39.538223,-95.146912
...,...,...,...,...,...,...,...,...,...
134503,134503,11001005001,DC,204439,0,0.079,0.000,38.911922,-77.029517
134504,134504,11001006804,DC,1542277,477741,0.595,0.184,38.886384,-76.970484
134505,134505,11001007000,DC,423352,0,0.163,0.000,38.881348,-76.995908
134506,134506,11001008100,DC,290283,0,0.112,0.000,38.893171,-76.992527


In [12]:
tracts_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134508 entries, 0 to 134507
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   copy_index   134508 non-null  int64  
 1   GEOID        134508 non-null  object 
 2   USPS         134508 non-null  object 
 3   ALAND        134508 non-null  int64  
 4   AWATER       134508 non-null  int64  
 5   ALAND_SQMI   134508 non-null  float64
 6   AWATER_SQMI  134508 non-null  float64
 7   INTPTLAT     134508 non-null  float64
 8   INTPTLONG    134508 non-null  float64
dtypes: float64(4), int64(3), object(2)
memory usage: 10.3+ MB


In [13]:
no_tracts = []

for index, i in enumerate(zip(tracts_list['copy_index'],tracts_list['USPS'])):
    if i[1] is np.nan:
        copy_index = i[0]
        USPS = i[1]
        no_tracts.append({'copy_index': copy_index, 
                       'USPS': USPS})

In [14]:
no_tracts

[]

In [15]:
no_tracts = pd.DataFrame(no_tracts, columns = ['copy_index', 'USPS'])
no_tracts

,copy_index,USPS


In [16]:
no_tracts

,copy_index,USPS


In [17]:
no_tracts = pd.merge(no_tracts, raw, on='copy_index')
no_tracts

,USPS,GEOID,Max,Sum,house_yr_pct_earlier_1939,house_pct_occupied,house_pct_family_married,race_pct_black,worked_past_12_mo,heat_pct_fueloil_kerosene,...,house_pct_live_alone,educ_some_col_no_grad,house_pct_ownd_occupied,house_w_home_equity_loan,house_val_175K_200K,house_val_200K_250K,True_Labels,Predicted_Labels,Predicted_Probs,copy_index


In [18]:
tracts_list = tracts_list.dropna(axis= 0)
tracts_list

,copy_index,GEOID,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,0,27017070600,MN,977465236,10306005,377.401,3.979,46.600099,-92.882993
1,1,26115831500,MI,4202063,64039,1.622,0.025,41.931336,-83.398945
2,2,26139020900,MI,4699929,1325661,1.815,0.512,43.064721,-86.213325
3,3,42011001600,PA,250449,0,0.097,0.000,40.340507,-75.913125
4,4,20005081800,KS,35296180,780343,13.628,0.301,39.538223,-95.146912
...,...,...,...,...,...,...,...,...,...
134503,134503,11001005001,DC,204439,0,0.079,0.000,38.911922,-77.029517
134504,134504,11001006804,DC,1542277,477741,0.595,0.184,38.886384,-76.970484
134505,134505,11001007000,DC,423352,0,0.163,0.000,38.881348,-76.995908
134506,134506,11001008100,DC,290283,0,0.112,0.000,38.893171,-76.992527


In [19]:
raw

,GEOID,Max,Sum,house_yr_pct_earlier_1939,house_pct_occupied,house_pct_family_married,race_pct_black,worked_past_12_mo,heat_pct_fueloil_kerosene,educ_bachelors,house_pct_live_alone,educ_some_col_no_grad,house_pct_ownd_occupied,house_w_home_equity_loan,house_val_175K_200K,house_val_200K_250K,True_Labels,Predicted_Labels,Predicted_Probs,copy_index
0,#_270170706003,1.508296,3.016591,0.259459,0.694595,0.513619,0.015060,0.619217,0.058366,0.071283,0.268482,0.197556,0.898833,0.038961,0.025974,0.086580,0,0,0.305971,0
1,#_261158315002,0.000000,0.000000,0.000000,0.878641,0.483425,0.004278,0.435175,0.000000,0.057007,0.334254,0.061758,0.762431,0.032609,0.047101,0.083333,0,0,0.128296,1
2,#_261390209002,0.000000,0.000000,0.804455,0.727723,0.489796,0.000000,0.647368,0.000000,0.493827,0.370748,0.080247,0.782313,0.108696,0.030435,0.130435,0,0,0.384859,2
3,#_420110016001,1.486989,1.486989,0.420147,0.916462,0.402145,0.023792,0.743702,0.335121,0.060820,0.193029,0.073550,0.512064,0.036649,0.000000,0.000000,0,0,0.350192,3
4,#_200050818003,2.210759,3.684598,0.735808,0.930131,0.570423,0.023581,0.646445,0.000000,0.256098,0.260563,0.131707,0.734742,0.019169,0.000000,0.000000,0,0,0.340038,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134503,#_110010050011,0.000000,0.000000,0.593137,0.926471,0.351852,0.070358,0.961735,0.000000,0.382843,0.365079,0.025608,0.497354,0.015957,0.000000,0.000000,0,0,0.000000,134503
134504,#_110010068041,0.000000,0.000000,0.851852,1.000000,0.296296,0.865349,0.487495,0.000000,0.037674,0.148148,0.116279,1.000000,0.148148,0.000000,0.000000,0,0,0.000000,134504
134505,#_110010070001,0.000000,0.000000,0.551389,0.948611,0.370425,0.078049,0.899194,0.000000,0.386930,0.387994,0.016337,0.522694,0.274510,0.000000,0.000000,0,0,0.000000,134505
134506,#_110010081003,0.000000,0.000000,0.895787,0.793792,0.455307,0.067775,0.751037,0.019553,0.431988,0.284916,0.017937,0.463687,0.156627,0.000000,0.000000,0,0,0.000000,134506


In [20]:
new_df = pd.merge(left=tracts_list, right=raw, how='left', left_on='copy_index', 
                   right_on='copy_index')

In [21]:
pd.set_option('display.max_columns', 500)

In [22]:
pd.set_option('display.max_rows', 500)

In [23]:
new_df

,copy_index,GEOID_x,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG,GEOID_y,Max,Sum,house_yr_pct_earlier_1939,house_pct_occupied,house_pct_family_married,race_pct_black,worked_past_12_mo,heat_pct_fueloil_kerosene,educ_bachelors,house_pct_live_alone,educ_some_col_no_grad,house_pct_ownd_occupied,house_w_home_equity_loan,house_val_175K_200K,house_val_200K_250K,True_Labels,Predicted_Labels,Predicted_Probs
0,0,27017070600,MN,977465236,10306005,377.401,3.979,46.600099,-92.882993,#_270170706003,1.508296,3.016591,0.259459,0.694595,0.513619,0.015060,0.619217,0.058366,0.071283,0.268482,0.197556,0.898833,0.038961,0.025974,0.086580,0,0,0.305971
1,1,26115831500,MI,4202063,64039,1.622,0.025,41.931336,-83.398945,#_261158315002,0.000000,0.000000,0.000000,0.878641,0.483425,0.004278,0.435175,0.000000,0.057007,0.334254,0.061758,0.762431,0.032609,0.047101,0.083333,0,0,0.128296
2,2,26139020900,MI,4699929,1325661,1.815,0.512,43.064721,-86.213325,#_261390209002,0.000000,0.000000,0.804455,0.727723,0.489796,0.000000,0.647368,0.000000,0.493827,0.370748,0.080247,0.782313,0.108696,0.030435,0.130435,0,0,0.384859
3,3,42011001600,PA,250449,0,0.097,0.000,40.340507,-75.913125,#_420110016001,1.486989,1.486989,0.420147,0.916462,0.402145,0.023792,0.743702,0.335121,0.060820,0.193029,0.073550,0.512064,0.036649,0.000000,0.000000,0,0,0.350192
4,4,20005081800,KS,35296180,780343,13.628,0.301,39.538223,-95.146912,#_200050818003,2.210759,3.684598,0.735808,0.930131,0.570423,0.023581,0.646445,0.000000,0.256098,0.260563,0.131707,0.734742,0.019169,0.000000,0.000000,0,0,0.340038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134503,134503,11001005001,DC,204439,0,0.079,0.000,38.911922,-77.029517,#_110010050011,0.000000,0.000000,0.593137,0.926471,0.351852,0.070358,0.961735,0.000000,0.382843,0.365079,0.025608,0.497354,0.015957,0.000000,0.000000,0,0,0.000000
134504,134504,11001006804,DC,1542277,477741,0.595,0.184,38.886384,-76.970484,#_110010068041,0.000000,0.000000,0.851852,1.000000,0.296296,0.865349,0.487495,0.000000,0.037674,0.148148,0.116279,1.000000,0.148148,0.000000,0.000000,0,0,0.000000
134505,134505,11001007000,DC,423352,0,0.163,0.000,38.881348,-76.995908,#_110010070001,0.000000,0.000000,0.551389,0.948611,0.370425,0.078049,0.899194,0.000000,0.386930,0.387994,0.016337,0.522694,0.274510,0.000000,0.000000,0,0,0.000000
134506,134506,11001008100,DC,290283,0,0.112,0.000,38.893171,-76.992527,#_110010081003,0.000000,0.000000,0.895787,0.793792,0.455307,0.067775,0.751037,0.019553,0.431988,0.284916,0.017937,0.463687,0.156627,0.000000,0.000000,0,0,0.000000


In [24]:
raw

,GEOID,Max,Sum,house_yr_pct_earlier_1939,house_pct_occupied,house_pct_family_married,race_pct_black,worked_past_12_mo,heat_pct_fueloil_kerosene,educ_bachelors,house_pct_live_alone,educ_some_col_no_grad,house_pct_ownd_occupied,house_w_home_equity_loan,house_val_175K_200K,house_val_200K_250K,True_Labels,Predicted_Labels,Predicted_Probs,copy_index
0,#_270170706003,1.508296,3.016591,0.259459,0.694595,0.513619,0.015060,0.619217,0.058366,0.071283,0.268482,0.197556,0.898833,0.038961,0.025974,0.086580,0,0,0.305971,0
1,#_261158315002,0.000000,0.000000,0.000000,0.878641,0.483425,0.004278,0.435175,0.000000,0.057007,0.334254,0.061758,0.762431,0.032609,0.047101,0.083333,0,0,0.128296,1
2,#_261390209002,0.000000,0.000000,0.804455,0.727723,0.489796,0.000000,0.647368,0.000000,0.493827,0.370748,0.080247,0.782313,0.108696,0.030435,0.130435,0,0,0.384859,2
3,#_420110016001,1.486989,1.486989,0.420147,0.916462,0.402145,0.023792,0.743702,0.335121,0.060820,0.193029,0.073550,0.512064,0.036649,0.000000,0.000000,0,0,0.350192,3
4,#_200050818003,2.210759,3.684598,0.735808,0.930131,0.570423,0.023581,0.646445,0.000000,0.256098,0.260563,0.131707,0.734742,0.019169,0.000000,0.000000,0,0,0.340038,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134503,#_110010050011,0.000000,0.000000,0.593137,0.926471,0.351852,0.070358,0.961735,0.000000,0.382843,0.365079,0.025608,0.497354,0.015957,0.000000,0.000000,0,0,0.000000,134503
134504,#_110010068041,0.000000,0.000000,0.851852,1.000000,0.296296,0.865349,0.487495,0.000000,0.037674,0.148148,0.116279,1.000000,0.148148,0.000000,0.000000,0,0,0.000000,134504
134505,#_110010070001,0.000000,0.000000,0.551389,0.948611,0.370425,0.078049,0.899194,0.000000,0.386930,0.387994,0.016337,0.522694,0.274510,0.000000,0.000000,0,0,0.000000,134505
134506,#_110010081003,0.000000,0.000000,0.895787,0.793792,0.455307,0.067775,0.751037,0.019553,0.431988,0.284916,0.017937,0.463687,0.156627,0.000000,0.000000,0,0,0.000000,134506


In [28]:
new_df['USPS'].unique()

array(['MN', 'MI', 'PA', 'KS', 'FL', 'AZ', 'VA', 'NY', 'NJ', 'IL', 'CA',
       'LA', 'IN', 'OK', 'CT', 'NC', 'TX', 'MD', 'GA', 'ND', 'MS', 'WA',
       'IA', 'SC', 'MA', 'OR', 'WY', 'DC', 'AR', 'AK', 'KY', 'OH', 'RI',
       'TN', 'WI', 'WV', 'NM', 'CO', 'NV', 'AL', 'NH', 'NE', 'ID', 'MO',
       'HI', 'DE', 'ME', 'VT', 'MT', 'UT', 'SD'], dtype=object)

In [32]:
new_df['USPS'].nunique()

51

In [25]:
#new_df = new_df.to_csv('PropensityBlockModel_FINAL.csv')